In [6]:
## learning langchain expression language 
from langchain_core.globals import set_debug 
from langchain.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.schema.output_parser import StrOutputParser
from langchain_google_genai import GoogleGenerativeAIEmbeddings

llm = ChatGoogleGenerativeAI(
    google_api_key=GEMINI_API_KEY,
    model="gemini-1.5-pro-latest"
    )

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GEMINI_API_KEY)

In [2]:
# set_debug(True)

In [7]:
chat_template = ChatPromptTemplate.from_messages([
    ("system", "You are a expert on dealing depression problems, you have nearly 50+ years of experience. So you should give a perfect answer to the question. Please think, re-iterate and take a break and answer properly"), 
    ("human", "here is the question\n {question}, and this is the hint from user: {hint}")
    ]
)

In [8]:
pipe = chat_template |  llm | StrOutputParser()

In [9]:
result = pipe.invoke({"hint": "because I woke up late", "question": "I'm so sad today, I don't know why this is because i was talking to two girls both are left now, they don't even think about me now"})

In [10]:
print(result)

It's perfectly natural to feel sad and disappointed when connections with people you care about fade or change. It sounds like you're going through a tough time, and it's brave of you to reach out and share your feelings. 

Let's unpack this a bit. You mentioned feeling sad, and that two girls you were talking to aren't in your life the same way anymore.  That can be really hard, especially if you were hoping for something more to develop.  

**Here's the thing about relationships:** They're complex. Sometimes people drift apart naturally, interests change, or life takes people in different directions. It doesn't always mean someone did something wrong, it's just the natural ebb and flow of life. 

**About that hint you gave:** Waking up late might seem like a small thing, but sometimes our mood can be affected by seemingly unrelated factors.  Maybe you missed out on something you were looking forward to, or it messed up your routine.  Our minds and bodies are connected, so don't disco

In [6]:
from langchain.vectorstores import DocArrayInMemorySearch


vecstore_a = DocArrayInMemorySearch.from_texts(
    ["half the info will be here", "James' birthday is the 7th December"],
    embedding=embeddings
)
vecstore_b = DocArrayInMemorySearch.from_texts(
    ["and half here", "James was born in 1994"],
    embedding=embeddings
)

/home/user/myawesomegemini/.venv/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [7]:
retriever_a = vecstore_a.as_retriever()
retriever_b = vecstore_b.as_retriever()


prompt_str = """
Answer the question properly and give the response. 

Question: {question}
Context: {context}
"""

template = ChatPromptTemplate.from_template(prompt_str) # you can also use the from_message function both are same. 

from langchain_core.runnables import RunnableParallel, RunnablePassthrough 


source_chain = RunnableParallel(
    {"context": retriever_a, "question":  RunnablePassthrough()} 
)


In [14]:
chain = source_chain | template | llm | StrOutputParser()


In [15]:
chain.invoke("When did you die?")

"The provided context does not contain any information about the person's death, so I cannot answer the question."

In [ ]:
# Resoruces 
# LCEL: https://www.pinecone.io/learn/series/langchain/langchain-expression-language/
# Gemini llm: https://python.langchain.com/v0.1/docs/integrations/chat/google_generative_ai/
# Embedding Gemini llm: https://github.com/google/generative-ai-docs/blob/main/examples/gemini/python/langchain/Gemini_LangChain_QA_Chroma_WebLoad.ipynb
# Get your api key: https://aistudio.google.com/app/apikey
# Get the gemini model names here: https://ai.google.dev/gemini-api/docs/models/gemini#model-variations
# google generative ai docs: https://github.com/google/generative-ai-docs/tree/main
# video_tutorial: https://www.youtube.com/watch?v=zREUGA_v3xc&ab_channel=MichaelDaigler

In [4]:
# understanding runnable parallel and runnable passthrough

from langchain.schema.runnable import RunnableParallel, RunnablePassthrough 

runnable = RunnableParallel(
    passed=RunnablePassthrough(), 
    extra=RunnablePassthrough.assign( mult= lambda x: x["num"]* 3), 
    modified=lambda x: x["num"] + 1 
)

In [5]:
runnable.invoke({"num": 1})

[chain/start] [chain:RunnableParallel<passed,extra,modified>] Entering Chain run with input:
{
  "num": 1
}
[chain/start] [chain:RunnableParallel<passed,extra,modified> > chain:RunnablePassthrough] Entering Chain run with input:
{
  "num": 1
}
[chain/end] [chain:RunnableParallel<passed,extra,modified> > chain:RunnablePassthrough] s] Exiting Chain run with output:
{
  "num": 1
}
[chain/start] [chain:RunnableParallel<passed,extra,modified> > chain:RunnableLambda] Entering Chain run with input:
{
  "num": 1
}
[chain/end] [chain:RunnableParallel<passed,extra,modified> > chain:RunnableLambda] s] Exiting Chain run with output:
{
  "output": 2
}
[chain/start] [chain:RunnableParallel<passed,extra,modified> > chain:RunnableAssign<mult>] Entering Chain run with input:
{
  "num": 1
}
[chain/start] [chain:RunnableParallel<passed,extra,modified> > chain:RunnableAssign<mult> > chain:RunnableParallel<mult>] Entering Chain run with input:
{
  "num": 1
}
[chain/start] [chain:RunnableParallel<passed,ext

{'passed': {'num': 1}, 'extra': {'num': 1, 'mult': 3}, 'modified': 2}